# Task 3 - Data Exploration

Task definition:
> Please summarize and compare both data sets statistically. Are there any patterns in the data that both share and are there others that differ between them? Can you make a recommendation for which type of contract would be more beneficial to trade in for Flexa?

With a bigger and more complex dataset I would probably start the exploration using a tool like `ydata-profiling` to get a quick overview of the data distribution and features. For the dataset we have here with effectivelly only one feature of interest, I will not use such tool and try to analyse using basic `pandas` and/or some simple statistical models.

As yo follow along the cells in this notebook, you will find my comments.


In [ ]:
import sys
import os
import logging
from pathlib import PurePath

# Configuration

In [ ]:
# add custom python modules root to the path variable, so that we can resuse code
root_path = PurePath(os.getcwd()).parents[1].joinpath("src")
if str(root_path) not in sys.path:
    sys.path.insert(0, str(root_path))
# sys.path

In [ ]:
%load_ext autoreload
%autoreload 2


logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logging.getLogger("matplotlib").setLevel(
    logging.ERROR
)  # Only show errors for this EDA since matplotlib is too verbose. Don't do this in production ;)

# Import dependencies

In [ ]:
from environs import Env
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Tuple
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

from scipy import signal

from datetime import datetime


from challenges.energy_trading.data_access import load_energy_data

myseed = 31
np.random.seed(myseed)
random.seed(myseed)

plt.rcParams.update({"figure.figsize": (20, 4), "figure.dpi": 200})
plt.style.use("seaborn-v0_8")

In [ ]:
env = Env()
env.read_env(".env", override=True)

dataset_path = env("ENERGY_TRADING_DATA")
dataset_path

# Data loading and prep


The 2 csv files we will load have a similar scheme. The first row represents the header and the following rows the entries for each respective periodicity, either hourly or every 15 minutes.

`"MTU (CET/CEST)","Day-ahead Price [EUR/MWh]","Currency","BZN|DE-LU"
"01.01.2022 00:00 - 01.01.2022 00:15","69.17","EUR"`

The last column named `BZN|DE-LU` is missing for all records in both files. I believe this column should contain information related to bidding zones in Germany and Luxembourg.

In [ ]:
def load_energy_data(filepath: str, suffix: str) -> pd.DataFrame:
    df = pd.read_csv(filepath + suffix, skiprows=1, names=["DateTime", "Price", "Currency"])
    return df


e_prices_raw_60m = load_energy_data(dataset_path, "_60min.csv")
e_prices_raw_15m = load_energy_data(dataset_path, "_15min.csv")

In [ ]:
e_prices_raw_60m

In [ ]:
e_prices_raw_15m

Both data sets are loaded but not cleaned/processed just yet.
Let's start with the data exploration to get a better understanding of the data we have at hand.

In [ ]:
e_prices_raw_60m.info()

In [ ]:
e_prices_raw_15m.info()

First thing to note is the column `DateTime` is not of date type, but rather a string (dtype `obj`). In order to simplify analysis, we will transform this column into two other columns that represent the time limits for this price: `start_time` and `end_time`.

We can also see some missing values for the `Price` and `Currency` columns in both data sets. Let's double-check this aspect.

In [ ]:
print(">>> 60min data:")
print(e_prices_raw_60m[e_prices_raw_60m.isnull().any(axis=1)])
print("-" * 90)
print("\n>>> 15min data:")
print(e_prices_raw_15m[e_prices_raw_15m.isnull().any(axis=1)])

As you can see, the entry `2042` correspoing to the timestamp `27.03.2022 02:00 - 27.03.2022 03:00` is missing the price and currency information in the `60min` data, and there are 4 records with missing data in the `15min` data set.

We should also check if all prices are specified in the same currency:

In [ ]:
print(">>> 60min data:")
print(e_prices_raw_60m["Currency"].unique())
print("-" * 90)
print("\n>>> 15min data:")
print(e_prices_raw_15m["Currency"].unique())

As expected, all records have either "EUR" or nan in the `Currency` column. Since our data represents information from bidding zone `BZN|DE-LU` and as stated in the data header, it is safe to assume `EUR` as the currency for all records and use it to handle the missing data in the `Currency` column.

We can handle missing values in several ways (called Imputation), which is totally dependent on the problem domain. Examples:
* Removing entries with missing values
* Forward fill or back fill: for time series data (like our prices data), it's common to use methods like forward filling (ffill) or backward filling (bfill) to fill in missing values based on nearby data points.
* Using statistical mthods: we could also fill missing values with the mean, median, or mode of the column, which might be appropriate depending on the distribution and the nature of the data. There are other more powerful algorithms to be considered too.
* Interpolation: estimates values using existing data points. Linear interpolation is common, but other methods (like polynomial or spline interpolation) can be more suitable depending on the nature of the time series.
* Regression Models: can estimate missing values.

Here is a **simplistic example** of a `RandomForestClassifier` to predict missing values:

```python
from sklearn.ensemble import RandomForestClassifier
# Split the data into sets with and without missing values
train_df = df[df['Price'].notna()]
test_df = df[df['Price'].isna()]

# Training a classifier
clf = RandomForestClassifier()
clf.fit(train_df.drop('Price', axis=1), train_df['Price'])

# Predicting the missing values
predicted_values = clf.predict(test_df.drop('Price', axis=1))
df.loc[df['Price'].isna(), 'Price'] = predicted_values
```

There are more sofisticated strategies, like the 'Multiple Imputation' approach where each missing value is imputed multiple times to generate a distribution of possible values, thus, better modeling/capturing the uncertainty around the true value. The `fancyimpute` python package provides some options to implement this strategy. 


For simplicity, in this example we will use `EUR` for missing data in the `Currency` column, and a combination of backward filling and median for the `Price` column. The idea here is the fall back to `median` in case the `bfill` fails.

But first, let's look at some descriptive statistics to get some insights on the data distribution and central tendencies of the data, and see if there are any obvious outliers, anomalies, or interesting patterns. The median, mean, standard deviation, minimum, maximum, and some percentiles of the energy prices can be valuable.  Then we can apply the imputation strategy and see how ti changes the data distribution.


In [ ]:
# a small set of statistic metrics to summarize the dataset

print(">>> 60min data:")
print(e_prices_raw_60m.describe())
print("-" * 90)
print("\n>>> 15min data:")
print(e_prices_raw_15m.describe())


What can we see in the `60min` data?
* The mean (average) price is approximately €185.80/MWh. This value gives you a central point of the data's distribution, suggesting that on average, the price per megawatt-hour across all observations is around this value.
* The standard deviation is about €90.66/MWh, indicating a high level of volatility in the price data. A high standard deviation in energy prices is common due to various factors affecting the market, such as changes in demand, fuel costs, weather conditions, and regulatory changes. The large value here tells you that the prices frequently vary a significant amount from the mean, highlighting the market’s unpredictability, which imposes an important challenge to predict a robust buy-sell strategy.
* The minimum price is -€19.04/MWh, which is quite unusual as it represents a **negative** price. I can imagine this could be a valid scenarion when there is an excess supply (e.g., high renewable generation) and not enough demand, making the producers to pay consumers to take the excess energy off the grid. This is also a good opportunity for consuming energy (buy it) to sell later at much higher price and thus, increaee profitability.
* The percentiles:
  * 25th (or first quartile) indicates that 25% of the prices are below €122.15/MWh. So only a quarter of the time on average we could benefit from much lower than average price.
  * 50% (Median) is €189.00/MWh, tells us that half the prices are above this value, and half are below. Thus, in 50% of the time we could sell above average price.
  * 75th (or third quartile) shows that 75% of the prices are below €232.94/MWh. Conversely, about 25% of the prices are higher than this value, which gives us 25% chance to sell at very good prices to increase profit.
* The maximum price observed is €700.00/MWh, highlighting the upper extreme of the market prices. This could be due to extraordinary demand or limited supply conditions (global crisis, extreme weather, or even local politics can affect the energy supply chain), and it marks scenarios where selling energy could be highly profitable.


What can we see in the `15min` data?
* the mean is very close to the mean and the percentiles in the `60min` data with just a small difference
* the variation in the `15min` data is much more pronounced, with standard deviation `13.49%` higher in the `15min` dataset.
* peaks and valeis are also more pronounced in ten `15min` data, repectively `max = 2999.99` and `min = -149.99`, which if exploited properly represent much better opportunities for increased profit.


**In summary**, the wide price variation and the high standard deviation suggest a highly volatile market with significant price fluctuations, which could provide opportunities for profit through timely buying and selling strategy. For now, due the the nature of high resolution and locality, the `15min` data presents slightly more oppotunities for profit. We still need to double-check this assumption.

In [ ]:
def imput_missing_data(df: pd.DataFrame) -> pd.DataFrame:
    # Check for any conversion issues
    if df["Price"].isnull().any():
        logging.warning("There are null values in the Price column")
        logging.warning("Imputing missing values with ffill.")
        df["Price"] = df["Price"].ffill()

        if pd.isna(df["Price"].iloc[0]):
            # Ensure no NaN values remain, if first value is NaN,
            # replace it with a sensible default (e.g., 0 or mean of the column)
            logging.warning("Imputing remaining missing values with median as fallback strategy.")
            df["Price"].iloc[0] = df["Price"].median()
    if df["Currency"].isnull().any():
        df["Currency"] = df["Currency"].fillna("EUR")
    return df

In [ ]:
e_prices_60m = imput_missing_data(e_prices_raw_60m)
e_prices_15m = imput_missing_data(e_prices_raw_15m)

Now we can check if all the missing data issues were solved:

In [ ]:
print(">>> 60min data:")
print(e_prices_60m.info())
print("-" * 90)
print("\n>>> 15min data:")
print(e_prices_15m.info())

Missing values are now handled in both datasets.

In [ ]:
print(">>> 60min data:")
print(e_prices_60m.describe())
print("-" * 90)
print("\n>>> 15min data:")
print(e_prices_15m.describe())

As expected there is not much change in the data distribution. We only had a handful of data points with missing data in a much bigger number of records, and thus, the imputation does not change the distribution significantly, which is exactly what we want.



Let's revisit the `DateTime` column. We need to split it into two other columns that represent the time limits for this price: `start_time` and `end_time`.



In [ ]:
def preprocess_data(df: pd.DataFrame, date_fmt: str = "%d.%m.%Y %H:%M") -> pd.DataFrame:
    """Split the 'DateTime' column into 'start_time' and 'end_time'"""
    datetime_splits = df["DateTime"].str.split(" - ", expand=True)
    df["start_time"] = pd.to_datetime(datetime_splits[0], format=date_fmt)
    df["end_time"] = pd.to_datetime(datetime_splits[1], format=date_fmt)

    return df


e_prices_60m = preprocess_data(e_prices_60m)
e_prices_15m = preprocess_data(e_prices_15m)

In [ ]:
print(">>> 60min data:")
print(e_prices_60m.head(5))
print("-" * 90)
print("\n>>> 15min data:")
print(e_prices_15m.head(5))

In [ ]:
print("Double-check if we did not add any null values and verify the dtypes are correctly set:\n")

print(">>> 60min data:")
print(e_prices_60m.info())
print("-" * 90)
print("\n>>> 15min data:")
print(e_prices_15m.info())

Let's verify we do have the expected number of data point in each dataset.

* `60min` data - since the dataset is covering 6 months (from January to June), we should expect to have `24 * number of days` data points. 
* `15min` data - in this case, we have 4 data points per hour, and thus, we expect `4 * 24 * number of days` data points.


In [ ]:
# Calculate the number of days between the two dates, inclusive of both start and end dates
start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 6, 30)
days_inclusive = (end_date - start_date).days + 1

In [ ]:
# 60min data
expected_data_points = days_inclusive * 24
first_timestamp = e_prices_60m["start_time"].min()
last_timestamp = e_prices_60m["start_time"].max()
assert e_prices_60m.shape[0] == expected_data_points

In [ ]:
# 15min data
expected_data_points = days_inclusive * 24 * 4
first_timestamp = e_prices_15m["start_time"].min()
last_timestamp = e_prices_15m["start_time"].max()
assert e_prices_15m.shape[0] == expected_data_points

In [ ]:
print(">>> 60min data:")
print(f"expected: {days_inclusive * 24} rows, got: {e_prices_60m.shape}")
print("-" * 90)
print("\n>>> 15min data:")
print(f"expected: {days_inclusive * 24 * 4} rows, got: {e_prices_15m.shape}")

## Visualizing the data

In [ ]:
# Auxiliary functions to generate the plots


def plot_df(x: pd.Series, y: pd.Series, title: str = "", xlabel: str = "Date", ylabel: str = "Value"):
    plt.figure(figsize=(16, 4))
    plt.fill_between(x, y1=y, alpha=0.5, linewidth=2, color="seagreen")
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.legend()
    plt.show()


def plot_mirrored(
    x: pd.Series,
    y: pd.Series,
    title: str = "",
    series_label: str = "",
    xlabel: str = "Date",
    ylabel: str = "Value",
    y_scale=800,
):
    fig, ax = plt.subplots(1, 1, figsize=(16, 4))
    plt.fill_between(x, y1=y, y2=-y, alpha=0.5, linewidth=2, color="seagreen", label=series_label)
    plt.ylim(-y_scale, y_scale)
    plt.title(title, fontsize=16)
    plt.gca().set(xlabel=xlabel, ylabel=ylabel)
    plt.hlines(y=0, xmin=np.min(x), xmax=np.max(x), linewidth=0.5)
    # plt.hlines(y=0, xmin=np.min(df[x_col_name]), xmax=np.max(df[x_col_name]), linewidth=.5)
    plt.legend()
    plt.show()


def plot_positive_negative(x: pd.Series, y: pd.Series, title: str = "", xlabel: str = "Date", ylabel: str = "Value"):
    plt.figure(figsize=(16, 4))

    # Plot positive values in one color and negative values in another
    plt.fill_between(x, y1=y, where=(y >= 0), alpha=0.5, linewidth=2, color="seagreen", label="Positive")
    plt.fill_between(x, y1=y, where=(y < 0), alpha=0.5, linewidth=2, color="salmon", label="Negative")

    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.legend()
    plt.show()


def plot_boxplot_series(
    df: pd.DataFrame, x_col_name: str, y_col_name: str, title: str = "Title", x_label: str = "", y_label: str = ""
):
    plt.figure(figsize=(16, 4))
    ax = sns.boxplot(x=x_col_name, y=y_col_name, data=df)

    # Set titles and labels
    ax.set_title(title, fontsize=20)
    ax.set_xlabel(x_label, fontsize=14)
    ax.set_ylabel(y_label, fontsize=14)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
    plt.show()


def plot_n_boxplot_series(df, target_columns: List[str], title: str, y_label: str, fig_size: Tuple = (32, 4)):
    # Number of target columns determines the number of subplots
    num_plots = len(target_columns)

    # Create a figure and a grid of subplots
    fig, axes = plt.subplots(nrows=num_plots, ncols=1, figsize=(fig_size[0], fig_size[1] * num_plots))

    # Flatten axes array if there's more than one subplot
    if num_plots > 1:
        axes = axes.flatten()
    else:
        axes = [axes]  # Ensure axes is iterable for a single subplot scenario

    # Plot each column on a separate subplot
    for i, column in enumerate(target_columns):
        sns.boxplot(x=column, y="Price", data=df, ax=axes[i], linewidth=2, color="skyblue")
        axes[i].set_title(f"{title} - {column}", fontsize=14)
        axes[i].set_xlabel(column)
        axes[i].set_ylabel(y_label)
        axes[i].legend(loc="best")

    # Adjust layout to prevent overlap
    plt.tight_layout()
    plt.show()


def plot_daily_aggregated_prices(df, title_prefix: str = ""):
    plt.figure(figsize=(16, 4))

    # Mean price line
    sns.lineplot(x="hour_of_day", y="mean", data=df, marker="o", label="Average Price")

    # Shaded area for standard deviation
    plt.fill_between(df["hour_of_day"], df["mean"] - df["std"], df["mean"] + df["std"], color="seagreen", alpha=0.3)

    plt.title(f"{title_prefix}Average Energy Prices by Hour of Day")
    plt.xlabel("Hour of Day")
    plt.ylabel("Average Price (EUR/MWh)")
    plt.xticks(range(0, 24))  # Ensure all hours are shown
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
plot_positive_negative(
    x=e_prices_60m.start_time, y=e_prices_60m.Price, title="60min - Openning price", ylabel="Price [EUR/MWh]"
)
plot_positive_negative(
    x=e_prices_15m.start_time, y=e_prices_15m.Price, title="15min - Openning price", ylabel="Price [EUR/MWh]"
)

In the `60min` data we can see a sharp increase in prices in mid February 2022 and peaked at around the mid March. That peak, which initially looked like a outlier clearly is not. It is part of the upward trend or energy prices increase that happened due to global economical situation.
The prices went down after that peak and returned to the normal variation until around mid June, 2022.

Also interesting to see the ~5 negative price occurencies, ploted as red lines.
At first sight, it doesn't look like there is any seasonality or strong trend. I looks rather random series with a high volatility as expected in pricing domain.

For the `15min` data, the situation is quite similar, but with more pronounced peaks and valeis. There seems to have a small number of extreme values around March, which could be a considered as outliers. However, givene the global events and crisis, this value could also be a legitimate value. I would digg in to external sources to validate this hypothesis and remove/replace these datapoints if they are indeed outliers.

Let's explore it deeper, since at this scale is difficult to see any seasonality.
I will check different level of aggregation windows, such as month, week, and day. For this we need to add some aggregated features/columns to the dataframes.

In [ ]:
def aggregate_timewindows(data: pd.DataFrame) -> pd.DataFrame:
    df = data.copy()
    df["month"] = df.start_time.dt.month
    df["day_of_month"] = df.start_time.dt.day
    df["week"] = df.start_time.dt.isocalendar().week
    df["day_of_week"] = df.start_time.dt.isocalendar().day
    # We could also filter out data if necessary to better understand specific periods.
    # Example:
    # expanded_df = expanded_df[(expanded_df['month'].isin([1, 2, 3])) & (expanded_df['week'] < 53)]
    return df

In [ ]:
expanded_60min_df = aggregate_timewindows(e_prices_60m)
expanded_60min_df["week"].unique()

In [ ]:
expanded_15min_df = aggregate_timewindows(e_prices_15m)
expanded_15min_df["week"].unique()

In [ ]:
print(">>> 60min data:")
print(expanded_60min_df.Price.isna().any())
print("-" * 90)
print("\n>>> 15min data:")
print(expanded_15min_df.Price.isna().any())

Note that first week is in the list above is 52. This is because January 1st, 2022, was part of the last week of 2021 according to the ISO week date system.

In [ ]:
plot_n_boxplot_series(
    expanded_60min_df,
    ["week", "month", "day_of_month", "day_of_week"],
    title="60min - Price Distribution Over 6 Months accross different periodicities",
    y_label="Price [EUR/MWh]",
)

In [ ]:
plot_n_boxplot_series(
    expanded_15min_df,
    ["week", "month", "day_of_month", "day_of_week"],
    title="15min - Price Distribution Over 6 Months accross different periodicities",
    y_label="Price [EUR/MWh]",
)


These charts still don't show much patterns.
Let's try to answer the following question:

> Is there any correlation of prices with hour of the day?

In [ ]:
expanded_60min_df["hour_of_day"] = expanded_60min_df["start_time"].dt.hour
hourly_prices_60min_df = expanded_60min_df.groupby("hour_of_day")["Price"].agg(["mean", "std"]).reset_index()

expanded_15min_df["hour_of_day"] = expanded_15min_df["start_time"].dt.hour
hourly_prices_15min_df = expanded_15min_df.groupby("hour_of_day")["Price"].agg(["mean", "std"]).reset_index()

In [ ]:
plot_daily_aggregated_prices(hourly_prices_60min_df, title_prefix="60min - ")
plot_daily_aggregated_prices(hourly_prices_15min_df, title_prefix="15min - ")

Now we can start to see some patterns, in both datasets. Around 5 AM the prices start to raise, reaching peak around 7 and 8 AM, lowering until around 2 PM and starting a new cycle. This looks like a senoid with two peaks per day, at the 7th hour (7 AM and 7 PM).

Thus, that is the moment to sell energy and realise the profit with respect to buys made around 3-4 hours earlier in the day, when prices are a lowes level.

Let's also plot a full day of each dataset just for curiosity.

In [ ]:
def plot_hourly_prices(day_data, title: str = "", periodicity: str = "Hourly", color="seagreen"):
    plt.figure(figsize=(16, 4))
    plt.plot(day_data["start_time"], day_data["Price"], marker="o", linestyle="-", color=color, alpha=0.7)
    plt.title(title)
    plt.xlabel(periodicity)
    plt.ylabel("Price (EUR/MWh)")
    plt.xticks(day_data["start_time"], day_data["start_time"].dt.time, rotation=90)

    plt.grid(True)
    plt.show()


target_date = pd.to_datetime("2022-05-05").date()
plot_hourly_prices(expanded_60min_df[expanded_60min_df["start_time"].dt.date == target_date], "Hourly prices")
plot_hourly_prices(
    expanded_15min_df[expanded_15min_df["start_time"].dt.date == target_date], "Quarter hour prices", color="orange"
)

The price volatility within 1 hour is also very high (as expected for financial market). Which again brings more opportunities for making profit with a well chosen and timely buy-vs-sell action.

I tend to believe this variation is even more pronounced in lower granularity.

## Decomposing the time series

> NOTE: in this part I'm going to speedup a bit due to time constraints.

Any time series can be seen as a combination of:
- base level
- trend: the longer-term movement in the data
- seasonality: the refular pattern withing a fixed period
- irreducible (residual) error: remainder of the data after trend and seasonal components have been removed

There is also a difference between `additive` and `multiplicative` time series, which represent the nature of the seasonal effect and how to these components are combined. If the seasonal effect varies with the level of the time series (e.g., higher prices lead to proportionally larger seasonal swings), we should use multiplicative.

**Additive**: `base` + `trend` + `seasonality` + $\epsilon$

**Multiplicative**: `base` * `trend` * `seasonality` * $\epsilon$


The `seasonal_decompose` in `statsmodels` implements the classical decomposition of a time series by considering the series as an additive or multiplicative combination of these components. However, multiplicative decomposition is not appropriate for zero and negative values, and therefore, we need to somehow change the negative prices we have in our timeseries if we want to use multiplicative decomposition.

There are several strategies to deal with this transformation, for example:

* Offsetting the Data: offset the entire dataset so that all values become positive. This is done by adding a constant to all data points in the series that is at least as large as the absolute value of the most negative number.
* Log Transformation: If the data can logically scale logarithmically (which implies no zero values), applying a log transformation can also help manage negative values. This approach is useful particularly for multiplicative models where exponential growth or decay is expected.


For simplicity, I will offset the data by a small constant shift which is just aa bit greater than the smallest negative number or zero.



In [ ]:
# Here we ensure the dataframe has a datetime index, which is required for time series analysis using statsmodels.
expanded_60min_df.set_index("start_time", inplace=True)
expanded_15min_df.set_index("start_time", inplace=True)

In [ ]:
print(expanded_60min_df.index)
print(expanded_15min_df.index)

In [ ]:
# lets review the datasets
expanded_60min_df

In [ ]:
expanded_15min_df

In [ ]:
# Additive Decomposition
# Here period defines the number of data points in each cycle.
# For hourly data, we use 24 data points per day and 96 for 15-minute data (4 points per hour).
result_add_60min = seasonal_decompose(expanded_60min_df["Price"], model="additive", period=24, extrapolate_trend="freq")
result_add_60min

In [ ]:
result_add_15min = seasonal_decompose(
    expanded_15min_df["Price"], model="additive", period=24 * 4, extrapolate_trend="freq"
)
result_add_15min

In [ ]:
plt.rcParams.update({"figure.figsize": (20, 5)})
result_add_60min.plot().suptitle("60min - Additive Decompose")
result_add_15min.plot().suptitle("15min - Additive Decompose")
plt.show()

In [ ]:
"""
Multiplicative Decomposition 
`extrapolate_trend` takes care of any missing values 
in the trend and residuals at the beginning of the series.
"""
print(f"offset_60min = {abs(expanded_60min_df["Price"].min()) + 0.01}")
print(f"offset_15min = {abs(expanded_15min_df["Price"].min()) + 0.01}")

# any zero price element?
expanded_15min_df[expanded_15min_df["Price"] == 0]
# No, therefore, we could also use log transformation.
# But lets keep it simple since we only want to understand if there are trends and seasonality for now.

In [ ]:
offset_60min = abs(expanded_60min_df["Price"].min()) + 0.01  # offseting by the smallest value + 1 cent of EUR
expanded_60min_df["Price_offset"] = expanded_60min_df["Price"] + offset_60min
expanded_60min_df[["Price", "Price_offset"]]

In [ ]:
offset_15min = abs(expanded_15min_df["Price"].min()) + 0.01  # offseting by the smallest value + 1 cent of EUR
expanded_15min_df["Price_offset"] = expanded_15min_df["Price"] + offset_15min
expanded_15min_df[["Price", "Price_offset"]]

In [ ]:
# double-check if there are negative prices in the offset column
print(expanded_60min_df["Price_offset"].min())
print(expanded_15min_df["Price_offset"].min())

In [ ]:
result_mul_60min = seasonal_decompose(
    expanded_60min_df["Price_offset"], model="multiplicative", period=24, extrapolate_trend="freq"
)
result_mul_15min = seasonal_decompose(
    expanded_15min_df["Price_offset"], model="multiplicative", period=96, extrapolate_trend="freq"
)

In [ ]:
plt.rcParams.update({"figure.figsize": (20, 5)})
result_mul_60min.plot().suptitle("60min - Multiplicative Decompose", fontsize=8)
plt.rcParams.update({"figure.figsize": (30, 10)})
result_mul_15min.plot().suptitle("15min - Multiplicative Decompose", fontsize=8)
plt.show()

Observations about the time series decomposition:

* no obvious trend, with the exception of mid-February to mid-March as already observed
* the seasonal component exhibits clear and consistent patterns, which would be even more clear at a higher resolution. This chart suggests a regular daily seasonality in the data, where specific hours of the day are systematically associated with higher or lower prices.
* The amplitude (height of the peaks) of the seasonal pattern appears relatively stable, indicating that the magnitude of these daily price fluctuations is consistent over the period analyzed. The amplitude of the seasonal cycles in the 15-minute data is more pronounced, suggesting that the finer granularity captures more intraday price fluctuations.
* The residuals are spread out with no clear pattern. There are some spikes on the residual which could generaly be considered outliers or events that the model did not capture (unusual market activities, demand spikes, or errors in data recording). The presence of these spikes in the residuals could indicate a need for models that can handle outliers or for further investigation to ensure data quality.
* The 15-minute data captures more detailed fluctuations, which could be important for short-term trading strategies or operational decisions for energy storage and generation.
* The larger spikes in the 15-minute residuals may warrant investigation to determine if they are due to actual market events or anomalies in data recording.
* While the 60-minute interval data provides a clearer, smoother trend, the 15-minute data captures more detailed variations which might be useful for high-frequency trading algorithms or detailed operational planning.
* Both datasets display a similar overall trend and daily seasonality, which indicates these patterns are significant.

Let's plot one more visualization to support the observation that no trend is actually there.



In [ ]:
detrended = pd.DataFrame()
detrended["detrended"] = expanded_60min_df["Price"] - result_mul_60min.trend
detrended["Price"] = expanded_60min_df["Price"]
detrended["detrended_lsf"] = signal.detrend(expanded_60min_df["Price"])
detrended

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
detrended["Price"].plot(label="Original")
detrended["detrended"].plot(label='Subtracted the "least squares fit"')
detrended["detrended_lsf"].plot(label='Subtracted the "trend component"')
ax.legend()
plt.title("Detrended series", fontsize=12)

After removing the trend, you're left with the seasonal and irregular components. Any patterns or cyclical behaviors that remain are due to seasonal effects or other non-trend related factors.
The residuals (the detrended prices) fluctuate around zero without a clear pattern or direction, this again suggests a non-pronounced trend component in this dataset.

In [ ]:
deseasonalized = pd.DataFrame()
deseasonalized["deseasonalized"] = expanded_60min_df["Price"] / result_mul_60min.seasonal
deseasonalized["Price"] = expanded_60min_df["Price"]

fig, ax = plt.subplots(figsize=(10, 5))
deseasonalized["Price"].plot(label="Original")
deseasonalized["deseasonalized"].plot(label="Seasonality removed")
ax.legend()

plt.title("Deseasonalized prices", fontsize=12)

## Stationary VS non-stationary

Since I'm planing on training different models for forecasting, especially ARIMA (AutoRegressive Integrated Moving Average) or its variations, checking for stationarity is a crucial. On fact, most statistical forecasting methods are based on the assumption that the time series are stationary.

In this domain (or finance data in general), one common problem of time series is they are not stationary, which means their statistical properties (mean, variance, maximal and minimal values) change over time. This can be verified with augmented [Dickey-Fuller test](https://en.wikipedia.org/wiki/Dickey%E2%80%93Fuller_test). 

Since several forecasting methods require a stationary series, we need to transform the series somehow to make it stationary. For example:

- `difference` the series at least until it become stationary. 
- take the log of the series if it has non-constant variance.
- take the `n-th` root of the series.
- fit some type of curve to the data and then model the residuals from that fit, if the data contain a trend.
- combination of the above methods


To verify if a series is stationary you can use `unit root` tests, like:
- visual inspection
- rolling statistics
- statiscal test (Augmented Dickey-Fuller Test (ADF), Kwiatkowski-Phillips-Schmidt-Shin (KPSS) Test, or Phillips-Perron Test (PP))
- Autocorrelation Function (ACF) and Partial Autocorrelation Function PACF plots, to check the correlation between a data point and its lagged values, which allows us to understand how past values influence future values


I will use the last option here.


In [ ]:
adf_result = adfuller(expanded_60min_df["Price"].values)
print(f"ADF Statistic: {adf_result[0]:.2f}")
print(f"p-value: {adf_result[1]:.5f}")
print("Critical Values:")
for key, value in adf_result[4].items():
    print(f"\t{key} {value:.3f}")

In [ ]:
adf_result = adfuller(expanded_15min_df["Price"].values)
print(f"ADF Statistic: {adf_result[0]:.2f}")
print(f"p-value: {adf_result[1]:.5f}")
print("Critical Values:")
for key, value in adf_result[4].items():
    print(f"\t{key} {value:.3f}")

In [ ]:
# first we need to make the data stationary
# this is the current dataset

print(expanded_60min_df[expanded_60min_df["Price"] == 0])  # <--- we still have some zero values to keep in mind

# In order to be able to apply log transformation, I need to either remove zeros or transform them.
# In the context of energy prices, a zero might indicate a lack of trading or other market anomalies
# rather than an actual price of zero. Therefore, one needs to be careful with this transformations.
# I will apply a small offset of 0.01 in the price column for the log transformations.

In [ ]:
expanded_15min_df
print(expanded_15min_df[expanded_15min_df["Price"] == 0])  # <--- No zero price in this dataframe

In [ ]:
# let's try several alternative, since we are exploring the data ... it does not cost us anything ;)
offset_60min = abs(expanded_60min_df["Price"].min()) + 0.01  # offseting by the smallest value + 1 cent of EUR
offset_15min = abs(expanded_15min_df["Price"].min()) + 0.01  # offseting by the smallest value + 1 cent of EUR

# differencing
expanded_60min_df["Price_diff"] = expanded_60min_df["Price"].diff().dropna()
expanded_15min_df["Price_diff"] = expanded_15min_df["Price"].diff().dropna()

# # log-transformed price, since we have no zeros in the Price column
expanded_60min_df["Price_log"] = np.log(
    expanded_60min_df["Price"] + offset_60min
)  # handling zeros: use a small offset constant
expanded_15min_df["Price_log"] = np.log(expanded_15min_df["Price"] + offset_15min)

# # linear detrending
expanded_60min_df["Price_detrended"] = signal.detrend(expanded_60min_df["Price"])
expanded_15min_df["Price_detrended"] = signal.detrend(expanded_15min_df["Price"])

# # Seasonal differencing
expanded_60min_df["Price_seasonal_diff"] = expanded_60min_df["Price"].diff(
    24
)  # assuming 24 observations per day for daily seasonality
expanded_15min_df["Price_seasonal_diff"] = expanded_15min_df["Price"].diff(
    96
)  # assuming 4*24 observations per day for daily seasonality

# # Combining Transformations:
expanded_60min_df["Price_log_diff"] = np.log(expanded_60min_df["Price"] + offset_60min).diff().dropna()
expanded_15min_df["Price_log_diff"] = np.log(expanded_15min_df["Price"] + offset_15min).diff().dropna()

In [ ]:
def plot_acf_pacf(series: pd.Series, lags: int, title: str = ""):
    plt.figure(figsize=(14, 7))
    plt.subplot(211)  # 2 rows, 1 column, 1st subplot
    plot_acf(series, ax=plt.gca(), lags=lags)  # Change lags to adjust the number of lags you want to see.
    plt.title(f"{title} - Autocorrelation Function")

    # Plot the Partial Autocorrelation Function (PACF)
    plt.subplot(212)  # 2 rows, 1 column, 2nd subplot
    plot_pacf(series, ax=plt.gca(), lags=lags)  # Change lags to adjust the number of lags you want to see.
    plt.title(f"{title} - Partial Autocorrelation Function")

    plt.tight_layout()
    plt.show()

In [ ]:
expanded_15min_df

The price related columns are: `Price`, `Price_offset`, `Price_diff`, `Price_log`, `Price_detrended`, `Price_seasonal_diff`, and `Price_log_diff`.

Remember `Price` column is not stationary.

* `Price_diff`: This is the first-difference of the original series. It typically works well for removing overall trends, making it good for identifying the MA component (q) of an ARIMA model.
* `Price_log_diff`: This is the first-difference of the log-transformed series. The log transformation can help stabilize the variance across the series, making this good for financial time series data, which often exhibits exponential growth. Not necessarily fit to our small dataset since no exponential growth is present in this sample.
* `Price_detrended`: This series has had a trend component removed, which is useful for highlighting the cyclical nature that might be obscured by the trend.
* `Price_seasonal_diff`: This is the seasonal difference of the original series. If there is strong seasonality, this can help to isolate the seasonal patterns for more granular analysis. However, the first cycles in the transformed data will be null due to the nature of the diff transformation. Thus, we will have to remove them or imput the values.
* `Price_log`: If you've taken the log of the series to stabilize the variance, this series will help you see relationships that are multiplicative in the original scale.


For seasonality, `Price_seasonal_diff` might be revealing, while `Price_diff` or `Price_log_diff` are useful if we are to traing a ARIMA model.

In [ ]:
from pandas.plotting import autocorrelation_plot

autocorrelation_plot(expanded_60min_df["Price"])
plt.show()

In [ ]:
autocorrelation_plot(expanded_15min_df["Price"])
plt.show()

In [ ]:
# the lags parameter to match the periodicity of the dataset
plot_acf_pacf(expanded_60min_df["Price_diff"][1:], lags=24, title="60min | Price_diff ")
plot_acf_pacf(expanded_60min_df["Price_log_diff"][1:], lags=24, title="60min | Price_log_diff ")
plot_acf_pacf(expanded_60min_df["Price_seasonal_diff"].dropna(), lags=24, title="60min | Price_seasonal_diff ")

Notes:
* the presence of seasonality would yield high autocorrelation.
* most data points fall outside the shaded area indicating autocorrelation with a strong statistical significance.
* slow decay in the ACF plot indicates that the series has a long memory. Sharp drops might indicate a seasonal pattern.
* A sharp cut-off after a few lags in the PACF plot suggests an AR (Autoregressive) process. Again, periodic spikes might point to seasonality.

In [ ]:
plot_acf_pacf(expanded_15min_df["Price_diff"][1:], lags=96, title="60min | Price_diff ")
plot_acf_pacf(expanded_15min_df["Price_log_diff"][1:], lags=96, title="60min | Price_log_diff ")
plot_acf_pacf(expanded_15min_df["Price_seasonal_diff"].dropna(), lags=96, title="60min | Price_seasonal_diff ")

In [ ]:
expanded_60min_df.info()